## 1. Cargar los datos

#### 1.1. Importar librerias y módulos

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from warnings import simplefilter
from importlib.machinery import SourceFileLoader
from sklearn.feature_selection import VarianceThreshold
import xlsxwriter

In [2]:
fun = SourceFileLoader( 'funciones', r'..\..\..\code\modules\funciones.py' ).load_module()
vn  = SourceFileLoader( 'variables_nombres', r'..\..\..\code\modules\variables_nombres.py' ).load_module()

In [3]:
simplefilter( action = "ignore", category = pd.errors.PerformanceWarning )
simplefilter( action = "ignore", category = FutureWarning )
# print( dir( fun ) )
# pd.set_option( 'display.max_rows', 10 )
# pd.set_option( 'display.max_columns', 10 )

#### 1.2. Importar bases 

Se importan las cuatro bases de manera conjunta, y se agrupan en la lista "bases". Para accerder a cada una de las bases se llama a la lista de bases + su respectivo índice. Ejemplo: bases[0]

In [4]:
pwd

'C:\\Users\\dell\\Documents\\GitHub\\Corruption_Paper\\code\\data\\preprocess_data'

In [9]:
path = r'..\..\..\input\built_data\matrix_c1_siaf.dta'

In [22]:
%%time

df = pd.read_stata( path )

CPU times: total: 14.6 s
Wall time: 14.9 s


#### 1.3. Última limpieza a las bases

Últimas modificaciones a las bases de datos

In [19]:
for col in df.columns:
    if "Sí" in df[ col ].values:
        print( col )

corrup_intensa
corrup_amplia


In [26]:
df[ 'corrup_intensa' ].value_counts()

corrup_intensa
1.0    1331
0.0     731
Name: count, dtype: int64

In [25]:
data_cols        = [ col for col in df.columns if col not in vn.no_predictoras_variables and col not in vn.string_variables ]
df               = df[ data_cols ]
encode_variables = { "corrup_intensa": { "Sí": 1, "No": 0 },  "corrup_amplia": { "Sí": 1, "No": 0 } }
df               = df.replace( encode_variables )
df               = df.astype( str ).replace( '', np.nan, regex = True )
df               = df.astype( str ).replace( '\.+$', np.nan, regex = True ).astype( float )

In [28]:
df.shape

(2062, 17539)

## 2. Preprocesamiento

### 2.1 Imputación con 0

Se imputa los valores perdidos de las variables de SIAF con 0. No se generan variables dummy de control.

In [29]:
df = fun.imputar_i( df, vn.siaf_variables, 0, dummy = False )

Se verifica que las variables pertenecientes a SIAF no tengan valores perdidos

In [34]:
df.isnull().sum().sum()

0

### 2.2. Filtro por variabilidad

Se genera una lista de variables que no deben ser modificadas o filtradas

In [31]:
vars_no_mod      = [ 'ubigeo', 'year', 'variable_gasto' ]
vars_no_mod_full = vn.dependientes_variables + vars_no_mod

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [32]:
df = fun.filtro_variabilidad( df, 0, vars_no_mod_full )

In [33]:
df.shape

(2062, 14740)

### 2.3. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF

In [35]:
df = fun.imputar_outliers( df, vn.siaf_variables, 0.01 )

### 2.7. Transformaciones logarítmicas

Se identifica las variables negativas de SIAF y se les divide entre un millón. 

In [36]:
df = fun.dividir_variables_negativas( df, 1_000_000, vars_no_mod_full )

Se crea una lista de variables a modificar logarítmicamente

In [37]:
num_predictores = vn.siaf_variables + vn.dependientes_numericas

Se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF que no son negativas.

In [38]:
df = fun.transformacion_log( df, num_predictores, vars_no_mod )

Se verifica el numero de variables por fuente de cada base luego del paso 2.7.

In [39]:
df.shape

(2062, 14740)

### 2.7. Exportar las bases de datos

In [40]:
df.to_csv( r'..\..\..\input\preprocessed_data\base0_siaf.csv', index = False )